<a href="https://colab.research.google.com/github/shubhi/msai-490-ai-industry-practicum/blob/main/week4-weights%26biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00


In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [5]:
diamonds.shape

(53940, 10)

In [6]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [8]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:04:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [9]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:04:32] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [10]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [11]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:04:36] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[20]	train-rmse:491.51435	validation-rmse:544.08058
[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [12]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:04:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[103]	train-rmse:375.44970	validation-rmse:532.50220


In [13]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:04:40] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [14]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [15]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [16]:
# Log in to your W&B account
import wandb

In [17]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see

In [18]:
sweep_config = {
    'method': 'bayes',  # Using Bayesian optimization
    'metric': {
        'name': 'rmse',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.01,
            'max': 0.1
        },
        'max_depth': {
            'distribution': 'int_uniform',
            'min': 3,
            'max': 9
        },
        'subsample': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 0.9
        },
        'colsample_bytree': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 0.9
        },
        'n_estimators': {
            'distribution': 'int_uniform',
            'min': 100,
            'max': 300
        },
        'reg_alpha': {
            'distribution': 'uniform',
            'min': 0.0,
            'max': 1.0
        },
        'reg_lambda': {
            'distribution': 'uniform',
            'min': 0.0,
            'max': 1.0
        }
    }
}


In [19]:
sweep_id = wandb.sweep(sweep_config, project="xgboost_diamonds_sweep_v5", entity='shubhigupta2025')

Create sweep with ID: 70qp7pyh
Sweep URL: https://wandb.ai/shubhigupta2025/xgboost_diamonds_sweep_v5/sweeps/70qp7pyh


In [20]:
def train():
    # Initialize a W&B run
    run = wandb.init()

    # Access the hyperparameters through wandb.config
    config = wandb.config

    # Define the model
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': config.learning_rate,
        'max_depth': int(config.max_depth),
        'subsample': config.subsample,
        'colsample_bytree': config.colsample_bytree,
        'n_estimators': int(config.n_estimators),
        'eval_metric': 'rmse'
    }


    # Train the model
    model = xgb.train(params, dtrain, num_boost_round=config.n_estimators)

    # Evaluate the model
    predictions = model.predict(dtest)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    # Log metrics
    wandb.log({'rmse': rmse})

    run.finish()


In [21]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: j0r1dx7i with config:
wandb: 	colsample_bytree: 0.6487029679976578
wandb: 	learning_rate: 0.04425632624485143
wandb: 	max_depth: 6
wandb: 	n_estimators: 246
wandb: 	reg_alpha: 0.8142253560247309
wandb: 	reg_lambda: 0.4032885738617237
wandb: 	subsample: 0.8610179687262243
wandb: Currently logged in as: shubhigupta2025. Use `wandb login --relogin` to force relogin


rmse,▁
rmse,533.34659


wandb: Agent Starting Run: syscrzks with config:
wandb: 	colsample_bytree: 0.6033584665328677
wandb: 	learning_rate: 0.0441048878891554
wandb: 	max_depth: 8
wandb: 	n_estimators: 236
wandb: 	reg_alpha: 0.6129941551241375
wandb: 	reg_lambda: 0.07547624872366165
wandb: 	subsample: 0.6622373335421011


rmse,▁
rmse,534.11683


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ehohhczg with config:
wandb: 	colsample_bytree: 0.7566517277483122
wandb: 	learning_rate: 0.04838156745770726
wandb: 	max_depth: 7
wandb: 	n_estimators: 115
wandb: 	reg_alpha: 0.16520217519318592
wandb: 	reg_lambda: 0.766003599431076
wandb: 	subsample: 0.5771819861024324


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: pbjeu7xg with config:
wandb: 	colsample_bytree: 0.8704557710607872
wandb: 	learning_rate: 0.024533868933606404
wandb: 	max_depth: 9
wandb: 	n_estimators: 138
wandb: 	reg_alpha: 0.48588791060416303
wandb: 	reg_lambda: 0.3927732949260675
wandb: 	subsample: 0.5056358706298488


rmse,▁
rmse,562.66072


wandb: Agent Starting Run: 60mj2koh with config:
wandb: 	colsample_bytree: 0.8869385693151993
wandb: 	learning_rate: 0.05553307752739365
wandb: 	max_depth: 9
wandb: 	n_estimators: 221
wandb: 	reg_alpha: 0.468884644399385
wandb: 	reg_lambda: 0.3336011710716471
wandb: 	subsample: 0.5197394744398728


rmse,▁
rmse,531.86523


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Hyperparameter Optimization Sweep Report for XGBoost on Diamond Dataset

### Sweep Overview

The hyperparameter optimization was conducted using Weights & Biases with Bayesian optimization on the XGBoost model applied to a dataset of diamonds. The goal was to minimize the Root Mean Square Error (RMSE) by tuning several hyperparameters.

### Hyperparameter Space

The following hyperparameters were included in the sweep:

- **Learning Rate**: Uniformly distributed between 0.01 and 0.1.
- **Max Depth**: Integer values uniformly distributed between 3 and 9.
- **Subsample**: Uniform distribution between 0.5 and 0.9.
- **Colsample_bytree**: Uniform distribution between 0.5 and 0.9.
- **N_estimators**: Integer values uniformly distributed between 100 and 300.
- **Reg_alpha**: Uniform distribution between 0.0 and 1.0 (L1 regularization).
- **Reg_lambda**: Uniform distribution between 0.0 and 1.0 (L2 regularization).

### Results Summary

The Bayesian sweep conducted multiple runs, each testing a different combination of hyperparameters. Below are the summarized results of selected runs:

| Run ID       | RMSE       | Learning Rate | Max Depth | N_estimators | Reg_alpha | Reg_lambda | Subsample | Colsample_bytree |
|--------------|------------|---------------|-----------|--------------|-----------|------------|-----------|------------------|
| j0r1dx7i     | 533.34659  | 0.0443        | 6         | 246          | 0.814     | 0.403      | 0.861     | 0.649            |
| syscrzks     | 534.11683  | 0.0441        | 8         | 236          | 0.613     | 0.0755     | 0.662     | 0.603            |
| ehohhczg     | 535.50793  | 0.0484        | 7         | 115          | 0.165     | 0.766      | 0.577     | 0.757            |
| tv5yq7sk     | 530.3824   | 0.0524        | 7         | 248          | 0.803     | 0.597      | 0.884     | 0.588            |
| vyfswxmm     | 531.68613  | 0.0640        | 8         | 190          | 0.188     | 0.272      | 0.564     | 0.823            |
| 10mzyn1z     | 568.65554  | 0.0431        | 5         | 139          | 0.198     | 0.902      | 0.854     | 0.605            |
| 3fi303pd     | 534.70286  | 0.0533        | 9         | 243          | 0.943     | 0.414      | 0.811     | 0.667            |
| lsiyfmkw     | 542.57643  | 0.0867        | 7         | 275          | 0.964     | 0.387      | 0.873     | 0.593            |
| fzu8e5ub     | 529.76083  | 0.0471        | 9         | 221          | 0.487     | 0.365      | 0.563     | 0.841            |
| bbrihgdt     | 529.04779  | 0.0413        | 7         | 156          | 0.632     | 0.221      | 0.539     | 0.868            |
| pbjeu7xg     | 562.66072  | 0.0245        | 9         | 138          | 0.486     | 0.393      | 0.506     | 0.870            |
| 60mj2koh     | 531.86523  | 0.0555        | 9         | 221          | 0.469     | 0.334      | 0.520     | 0.887            |

### Analysis

- **Best Performing Configurations**: The best RMSE scores were found in the runs with Run IDs `fzu8e5ub` and `bbrihgdt`, producing RMSEs of 529.76083 and 529.04779 respectively. These configurations suggest that a combination of higher `colsample_bytree`, moderate `learning_rate`, and `n_estimators` in the mid-range tend to perform better.
- **Parameter Influence**: The `max_depth` and `subsample` values did not show a consistent trend, indicating that their optimal values may depend more on their interaction with other parameters like `reg_alpha` and `reg_lambda`.
- **Variability**: The RMSE scores fluctuated across different runs, which highlights the complexity of the hyperparameter interactions and the importance of fine-tuning each parameter.